BERIKUT UTK DOWNGRADE KE PYTHON 3.8.0

In [ ]:
!which python
!python --version

In [ ]:
%env PYTHONPATH=

In [ ]:
!pip install virtualenv

In [ ]:
!virtualenv myenv

In [ ]:
!wget https://repo.anaconda.com/miniconda/Miniconda3-py38_4.8.2-Linux-x86_64.sh

In [ ]:
!chmod +x Miniconda3-py38_4.8.2-Linux-x86_64.sh

In [ ]:
!./Miniconda3-py38_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
!conda install -q -y --prefix /usr/local python=3.8 ujson

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.8/site-packages/')

In [ ]:
import os
os.environ['CONDA_PREFIX'] = '/usr/local/envs/myenv'

In [ ]:
!python --version

BERIKUT PERSIAPAN TRAINING TENSORFLOW LITE

In [ ]:
!pip uninstall Cython -y # Temporary fix for "No module named 'object_detection'" error
!git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
import re
with open('/content/models/research/object_detection/packages/tf2/setup.py') as f:
    s = f.read()

with open('/content/models/research/setup.py', 'w') as f:
    # Set fine_tune_checkpoint path
    s = re.sub('tf-models-official>=2.5.1',
               'tf-models-official==2.8.0', s)
    f.write(s)

In [ ]:
!pip install pyyaml==5.3

In [ ]:
!pip install /content/models/research/

In [ ]:
!pip install tensorflow==2.8.0

In [ ]:
!pip install tensorflow_io==0.23.1

In [ ]:
!sudo apt-get remove -y protobuf-compiler

In [ ]:
!pip install 'protobuf<=3.20.1' --force-reinstall

In [ ]:
!wget https://github.com/protocolbuffers/protobuf/releases/download/v3.20.1/protoc-3.20.1-linux-x86_64.zip

In [ ]:
!unzip protoc-3.20.1-linux-x86_64.zip -d protoc3

In [ ]:
!sudo mv protoc3/bin/* /usr/local/bin/

In [ ]:
!sudo mv protoc3/include/* /usr/local/include

In [ ]:
# Install CUDA version 11.0 (to maintain compatibility with TF v2.8.0)
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget http://developer.download.nvidia.com/compute/cuda/11.0.2/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1804-11-0-local_11.0.2-450.51.05-1_amd64.deb
!apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
!apt-get update && sudo apt-get install cuda-toolkit-11-0
!export LD_LIBRARY_PATH=/usr/local/cuda-11.0/lib64:$LD_LIBRARY_PATH

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py